In [1]:
import pandas as pd
import numpy as np
import joblib

#Visualización
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

#Métricas
import sklearn as sk
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report, make_scorer

#Configuración Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#Regressors
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasRegressor

from keras import backend as K
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

#Análisis de Sentimientos
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD

2024-11-29 09:10:42.086069: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 09:10:42.089455: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 09:10:42.099617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732882242.116684  116087 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732882242.121725  116087 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 09:10:42.138472: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
user_stories_train = pd.read_csv('data/train.csv', low_memory=False)

In [3]:
user_stories_train.head()

,id,title,description,project,storypoint
0,5660,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,3
1,9014,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,5
2,4094,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,5
3,811,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,2
4,4459,transform processor with script option is broken,Creating the following stream throws exception...,project6,2


In [4]:
user_stories_train.dtypes

id              int64
title          object
description    object
project        object
storypoint      int64
dtype: object

In [5]:
# Vemos cantidad de filas y columnas
user_stories_train_size = user_stories_train.shape
print("Cantidad de columnas: %d" % user_stories_train_size[1])
print("Cantidad de filas: %d" % user_stories_train_size[0])

Cantidad de columnas: 5
Cantidad de filas: 7900


In [6]:
# Analizamos si hay filas duplicadas
user_stories_train_total = len(user_stories_train)
user_stories_train_unique = user_stories_train.drop_duplicates()
user_stories_train_unique_size = len(user_stories_train_unique)
print(f'Se eliminaron: {user_stories_train_total - user_stories_train_unique_size} filas duplicadas')

Se eliminaron: 0 filas duplicadas


In [7]:
# Vemos cantidad de datos faltantes
user_stories_train.isna().sum()

id             0
title          0
description    0
project        0
storypoint     0
dtype: int64

In [8]:
#Verifico balanceo de clases
user_stories_train['storypoint'].value_counts(normalize=True)*100

storypoint
3     23.392405
5     21.430380
1     20.620253
2     16.151899
8     12.974684
4      2.101266
13     1.949367
10     0.405063
20     0.392405
6      0.215190
16     0.088608
12     0.075949
40     0.063291
21     0.037975
15     0.025316
7      0.025316
32     0.012658
34     0.012658
14     0.012658
24     0.012658
Name: proportion, dtype: float64

### Preparamos el dataset

In [9]:
#Creamos un dataset con features a usar para clasificar
user_stories_x = user_stories_train['description'].copy()

#Creamos un dataset con la variable target 'storypoint'
user_stories_y = user_stories_train['storypoint'].copy()

#Genero los conjuntos de train y test
x_train, x_test, y_train, y_test = train_test_split(user_stories_x,
                                                   user_stories_y,
                                                   test_size=0.3,  #proporcion 70/30
                                                   random_state=2) #semilla

In [10]:
x_train

2646    The plugin will need to contribute the icon. W...
2724    The timer: component is used to generate messa...
5159    h5.Description:  In CLI we have the ability to...
976     We need to refactor the way how we package and...
2187    Standalone Admin currently has no shiny banner...
                              ...                        
3606    h5. Description:  When launching Studio with C...
5704    When a .tss file contains certain grammars, Ti...
6637    The data that is entering a broadcast stream c...
2575    Provide the infrastructure for HTTP GET /compl...
7336    Studio shows an Empty Preferences window when ...
Name: description, Length: 5530, dtype: object

In [11]:
y_train

2646    8
2724    2
5159    8
976     2
2187    3
       ..
3606    5
5704    2
6637    2
2575    3
7336    3
Name: storypoint, Length: 5530, dtype: int64

## TensorFlow y Keras

###Cross Validation

In [12]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/users/pablo.prieto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/users/pablo.prieto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/users/pablo.prieto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def lemmatizationAndStopwords(text):
    wordNetLemmatizer = WordNetLemmatizer()
    stopwordSets = set(stopwords.words('english'))
    words = word_tokenize(text)
    wordsFilter = [wordNetLemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stopwordSets and word.isalpha()]
    if not wordsFilter:
        return "empty"
    return ' '.join(wordsFilter)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

max_tokens = 20000
sequence_length = 100

def build_model(input_dim, dropout_rate=0.3, dense_units=64, learning_rate=0.01):
    model = Sequential([
        Dense(128, activation="relu", input_dim=input_dim),
        Dropout(dropout_rate),
        Dense(dense_units, activation="relu"),
        Dense(1, activation="linear")
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='mean_squared_error', 
                  metrics=['RootMeanSquaredError'])
    return model
    

In [15]:
keras_regressor = KerasRegressor(
    model=build_model,
    input_dim=100,
    dropout_rate=0.3,
    dense_units=64,
    learning_rate=0.01,
    epochs=20,
    batch_size=32,
    verbose=0
)

print(keras_regressor.get_params().keys())

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'input_dim', 'dropout_rate', 'dense_units', 'learning_rate'])


In [16]:
##KFOLD CV usando TensorFlow y Keras (los mejores atributos, hiperparametros,etc)
from sklearn.pipeline import Pipeline

parameters = {
    "tfidf__analyzer": ["word"],
    # "tfidf__smooth_idf": [True,False],
    # "tfidf__ngram_range": [(1, 1),(1, 2)],
    "tfidf__ngram_range": [(1, 2)],
    "tfidf__use_idf": [False],
    "tfidf__stop_words": [['english']],
    "tfidf__max_features": [1000, 2000, 3000],
    "tfidf__min_df": [2,3,5],
    # 'tfidf__max_df': [0.75, 0.8, 1.0],
    "tfidf__max_df": [0.8],
    "tfidf__sublinear_tf": [False],
    "svd__n_components": [100, 300, 500],
    "svd__random_state" :[42],
    "keras_regressor__dropout_rate": [0.2, 0.3, 0.4],
    "keras_regressor__dense_units": [32, 64, 128],
    "keras_regressor__learning_rate": [0.0001, 0.001, 0.01],
    "keras_regressor__batch_size": [16, 32, 64, 128],
    "keras_regressor__epochs": [10, 20, 30, 50]
}

randomcv_best_score = None

#Iteración Cantidad de splits para el Cross Validation
for folds in [10,11,12,13,14,15,16,17,18,20]:
# for folds in [6,7,8,9,10,11,12]:
# for folds in [5,10]:
    
    #Kfold
    kfoldcv = KFold(n_splits=folds, shuffle=True, random_state=42)

    #Regressors
    pipeline = Pipeline(
        steps=[
            ("tfidf", TfidfVectorizer(preprocessor=lemmatizationAndStopwords)),
            ('svd', TruncatedSVD()),
            ("keras_regressor", keras_regressor),
        ]
    )
    
    #Metrica que quiero optimizar root_mean_squared_error
    scorer_fn = make_scorer(root_mean_squared_error)

    #Random Search Cross Validation
    #Cantidad de combinaciones que quiero probar
    n=10
    randomcv_it = RandomizedSearchCV(estimator = pipeline,
                                     param_distributions = parameters,
                                     scoring = scorer_fn,
                                     cv = kfoldcv,
                                     n_iter = n,
                                     verbose=1, 
                                     n_jobs=1)

    # #Grid Search Cross Validation
    # randomcv_it = GridSearchCV(estimator = pipeline,
    #                                  param_grid = parameters,
    #                                  scoring = scorer_fn,
    #                                  cv = kfoldcv) 

    #Busco los hiperparamtros que optimizan F1 Score
    randomcv_it.fit(x_train,y_train);
    
    #Mejores hiperparametros del arbol
    print("folds: ", folds)
    print(randomcv_it.best_params_)
    #Mejor métrica
    print("root_mean_squared_error: ", randomcv_it.best_score_)
    print(" ")
    print("mean_test_score: ", randomcv_it.cv_results_['mean_test_score'])
    
    #Nos quedamos con el menor error root_mean_squared_error
    if randomcv_best_score is None:
        randomcv_best_score = randomcv_it
    elif randomcv_it.best_score_ < randomcv_best_score.best_score_:
        randomcv_best_score = randomcv_it

Fitting 10 folds for each of 10 candidates, totalling 100 fits


2024-11-29 09:10:50.835340: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
80 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packa

folds:  10
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.0001, 'keras_regressor__epochs': 10, 'keras_regressor__dropout_rate': 0.2, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 64}
root_mean_squared_error:  2.933193793877224
 
mean_test_score:  [2.88463017        nan        nan        nan        nan        nan
        nan 2.93319379        nan        nan]
Fitting 11 folds for each of 10 candidates, totalling 110 fits


/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
77 fits failed out of a total of 110.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packa

...

folds:  17
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 3, 'tfidf__max_features': 3000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 20, 'keras_regressor__dropout_rate': 0.2, 'keras_regressor__dense_units': 64, 'keras_regressor__batch_size': 64}
root_mean_squared_error:  3.004172626491987
 
mean_test_score:  [       nan        nan 2.85590419 3.00417263 2.82762787 2.84182737
        nan 2.99100651        nan        nan]
Fitting 18 folds for each of 10 candidates, totalling 180 fits


/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
108 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-pack

folds:  18
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 30, 'keras_regressor__dropout_rate': 0.4, 'keras_regressor__dense_units': 64, 'keras_regressor__batch_size': 32}
root_mean_squared_error:  2.8852178270957105
 
mean_test_score:  [       nan 2.85270253        nan 2.81016239 2.88521783        nan
 2.786675          nan        nan        nan]
Fitting 20 folds for each of 10 candidates, totalling 200 fits


/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
140 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/users/pablo.prieto/.local/lib/python3.12/site-pack

folds:  20
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.0001, 'keras_regressor__epochs': 10, 'keras_regressor__dropout_rate': 0.3, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 128}
root_mean_squared_error:  3.473329876120733
 
mean_test_score:  [       nan        nan        nan 2.80525217        nan        nan
        nan 2.91470142 3.47332988        nan]


In [17]:
randomcv_best_score.best_params_

{'tfidf__use_idf': False,
 'tfidf__sublinear_tf': False,
 'tfidf__stop_words': ['english'],
 'tfidf__ngram_range': (1, 2),
 'tfidf__min_df': 5,
 'tfidf__max_features': 2000,
 'tfidf__max_df': 0.8,
 'tfidf__analyzer': 'word',
 'svd__random_state': 42,
 'svd__n_components': 100,
 'keras_regressor__learning_rate': 0.01,
 'keras_regressor__epochs': 30,
 'keras_regressor__dropout_rate': 0.4,
 'keras_regressor__dense_units': 64,
 'keras_regressor__batch_size': 32}

RANDOM SEARCH CV

folds: 8 {'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.001, 'keras_regressor__epochs': 20, 'keras_regressor__dropout_rate': 0.4, 'keras_regressor__dense_units': 128, 'keras_regressor__batch_size': 64}


root_mean_squared_error:  2.8482858544739167



folds:  10
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.0001, 'keras_regressor__epochs': 10, 'keras_regressor__dropout_rate': 0.2, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 64}

root_mean_squared_error:  2.933193793877224


folds:  11
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 50, 'keras_regressor__dropout_rate': 0.4, 'keras_regressor__dense_units': 128, 'keras_regressor__batch_size': 16}

root_mean_squared_error:  2.9115522957463797



folds:  12
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.001, 'keras_regressor__epochs': 10, 'keras_regressor__dropout_rate': 0.4, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 16}


root_mean_squared_error:  2.8201966753545467


folds:  13
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.0001, 'keras_regressor__epochs': 10, 'keras_regressor__dropout_rate': 0.3, 'keras_regressor__dense_units': 64, 'keras_regressor__batch_size': 128}

root_mean_squared_error:  3.1614441921713823


folds:  14
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 30, 'keras_regressor__dropout_rate': 0.2, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 128}

root_mean_squared_error:  3.11053177214984


folds:  15
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 50, 'keras_regressor__dropout_rate': 0.3, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 64}

root_mean_squared_error:  3.0086830477900315


folds:  16
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 3, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 50, 'keras_regressor__dropout_rate': 0.2, 'keras_regressor__dense_units': 128, 'keras_regressor__batch_size': 128}

root_mean_squared_error:  3.0802109031893887


folds:  17
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 3, 'tfidf__max_features': 3000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 20, 'keras_regressor__dropout_rate': 0.2, 'keras_regressor__dense_units': 64, 'keras_regressor__batch_size': 64}

root_mean_squared_error:  3.004172626491987


folds:  18
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.01, 'keras_regressor__epochs': 30, 'keras_regressor__dropout_rate': 0.4, 'keras_regressor__dense_units': 64, 'keras_regressor__batch_size': 32}

root_mean_squared_error:  2.8852178270957105


folds:  20
{'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.0001, 'keras_regressor__epochs': 10, 'keras_regressor__dropout_rate': 0.3, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 128}

root_mean_squared_error:  3.473329876120733


In [18]:
randomcv_best_score.best_score_

np.float64(2.8852178270957105)

In [19]:
randomcv_best_score.cv_results_['mean_test_score']

array([       nan, 2.85270253,        nan, 2.81016239, 2.88521783,
              nan, 2.786675  ,        nan,        nan,        nan])

THE BEST: 

 {'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100, 'keras_regressor__learning_rate': 0.001, 'keras_regressor__epochs': 10, 'keras_regressor__dropout_rate': 0.4, 'keras_regressor__dense_units': 32, 'keras_regressor__batch_size': 16}

root_mean_squared_error: 2.8201966753545467